# Practice1. Statistical Graphs

### 1)Stack Bar

In [1]:
from pyecharts import options as opts
from pyecharts.charts import Bar,Timeline,Pie,Graph,Geo, Grid,Parallel
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode

# Data 
list2 = [    {"value": 12, "percent": 12 / (12 + 3)},
    {"value": 23, "percent": 23 / (23 + 21)},
    {"value": 33, "percent": 33 / (33 + 5)},
    {"value": 3, "percent": 3 / (3 + 52)},
    {"value": 33, "percent": 33 / (33 + 43)},
]
list3 = [
    {"value": 3, "percent": 3 / (12 + 3)},
    {"value": 21, "percent": 21 / (23 + 21)},
    {"value": 5, "percent": 5 / (33 + 5)},
    {"value": 52, "percent": 52 / (3 + 52)},
    {"value": 43, "percent": 43 / (33 + 43)},
]

# Initiate with fixed width and color scheme
Stackbar=Bar(init_opts=opts.InitOpts(width='630px',theme=ThemeType.LIGHT))

# Add xaxis and double yaxis
Stackbar.add_xaxis([1, 2, 3, 4, 5])
Stackbar.add_yaxis("product1", list2, stack="stack1", category_gap="50%")
Stackbar.add_yaxis("product2", list3, stack="stack1", category_gap="50%")

# Options for label position and string formmat
Stackbar.set_series_opts(
        label_opts=opts.LabelOpts(
            position="right",
            formatter=JsCode(
                "function(x){return Number(x.data.percent * 100).toFixed() + '%';}"
            ),
        )
    )

# Present on notebook
Stackbar.render_notebook()

### 2)Bar Options

In [2]:
# Initiate with fixed width
bar=Bar(init_opts=opts.InitOpts(width='630px'))

# Add xaxis and double yaxis
bar.add_xaxis(["Long item x label 1",
            "Long item x label 2",
            "Long item x label 3",
            "Long item x label 4",
            "Long item x label 5",
            "Long item x label 6",])
bar.add_yaxis("MerchantA", [10, 20, 30, 40, 50, 40])
bar.add_yaxis("MerchantB", [20, 10, 40, 30, 40, 50])

# Options for label rotation and title
bar.set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15)),
        title_opts=opts.TitleOpts(title="Bar-Rotate X Label", subtitle="Solotions for long item"),

    )

# Options for Markpoint
bar.set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="Maximum"),
                    opts.MarkPointItem(type_="min", name="Minimum"),
                    opts.MarkPointItem(type_="average", name="Averate"),
                ]
            )
    )

# Present on notebook
bar.render_notebook()

### 3)Multiple Dynamic Graphs

In [3]:
from pyecharts.faker import Faker
total_data={}
# Using Faker to generate data
name_list=Faker.country
data_gdp={}
data_pi={}
data_si={}
data_ti={}
data_estate={}
data_financial={}
for year in range(2002, 2012):
    data_gdp[year]=Faker.values()
    data_pi[year]=Faker.values()
    data_si[year]=Faker.values()
    data_ti[year]=Faker.values()
    data_estate[year]=Faker.values()
    data_financial[year]=Faker.values()

# Format data function
def format_data(data: dict) -> dict:
    for year in range(2002, 2012):
        max_data, sum_data = 0, 0
        temp = data[year]
        max_data = max(temp)
        for i in range(len(temp)):
            sum_data += temp[i]
            data[year][i] = {"name": name_list[i], "value": temp[i]}
        data[str(year) + "max"] = int(max_data / 100) * 100
        data[str(year) + "sum"] = sum_data
    return data
# GDP
total_data["dataGDP"] = format_data(data=data_gdp)
# Primary industry
total_data["dataPI"] = format_data(data=data_pi)
# Secondary industry
total_data["dataSI"] = format_data(data=data_si)
# Tertiary Industry
total_data["dataTI"] = format_data(data=data_ti)
# Estate
total_data["dataEstate"] = format_data(data=data_estate)
# Finance
total_data["dataFinancial"] = format_data(data=data_financial)

#####################################################################################
# 2002 - 2011 years' data
def get_year_overlap_chart(year: int) -> Bar:
    bar = (
        Bar()
        .add_xaxis(name_list)
        .add_yaxis("GDP",total_data["dataGDP"][year])
        .add_yaxis(
            "Finance",
            total_data["dataFinancial"][year]
        )
        .add_yaxis(
            "Estate",
            total_data["dataEstate"][year]
        )
        .add_yaxis(
            "Primary industry",
            total_data["dataPI"][year]
        )
        .add_yaxis(
           "Secondary industry",
            total_data["dataSI"][year]
        )
        .add_yaxis(
            "Tertiary Industry",
            total_data["dataTI"][year]
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False)
        )
        .set_global_opts(            
            title_opts=opts.TitleOpts(
                title="{}".format(year)
            ),
            tooltip_opts=opts.TooltipOpts(
                is_show=True, trigger="axis", axis_pointer_type="shadow"
            ),
        )
    )
    pie = (
        Pie()
        .add(
            series_name="GDP_Percent",
            data_pair=[
                ["Primary industry", total_data["dataPI"]["{}sum".format(year)]],
                ["Secondary industry", total_data["dataSI"]["{}sum".format(year)]],
                ["Tertiary Industry", total_data["dataTI"]["{}sum".format(year)]],
            ],
            center=["75%", "35%"],
            radius="28%",
        )
        .set_series_opts(tooltip_opts=opts.TooltipOpts(is_show=True, trigger="item"))
    )
    return bar.overlap(pie)


# Graph to generate timeline
timeline = Timeline(init_opts=opts.InitOpts(width="630px", height="500px"))

# Add data to timeline
for y in range(2002, 2012):
    timeline.add(get_year_overlap_chart(year=y), time_point=str(y))

# Autoplay Setting
timeline.add_schema(is_auto_play=True, play_interval=1000)

# Present on notebook
timeline.render_notebook()

### 4) 3DBar

In [4]:
from pyecharts.charts import Bar3D
import random

# Clear dataset
data=[]

# Generate random data
for i in range(24):
    for j in range(7):
        t=random.randint(0,15)
        data.append([i,j,t])
        
# Render the graph directly
(Bar3D(init_opts=opts.InitOpts(width='630px'))
    .add(
        series_name="",
        data=data,
        xaxis3d_opts=opts.Axis3DOpts(type_="category", data=Faker.clock),
        yaxis3d_opts=opts.Axis3DOpts(type_="category", data=Faker.week_en),
        zaxis3d_opts=opts.Axis3DOpts(type_="value"),
    )
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(
            max_=20,
            range_color=[
                "#313695",
                "#4575b4",
                "#74add1",
                "#abd9e9",
                "#e0f3f8",
                "#ffffbf",
                "#fee090",
                "#fdae61",
                "#f46d43",
                "#d73027",
                "#a50026",
            ],
        )
    ).render_notebook()
)     

### 4)Grid geo bar

In [5]:
# Using faker to generate bar
bar = (
    Bar()
    .add_xaxis(Faker.country)
    .add_yaxis("A", Faker.values())
    .add_yaxis("B", Faker.values())
    .set_global_opts(legend_opts=opts.LegendOpts(pos_left="20%"))
)

# Initiate map
geo = (
    Geo()
    .add_schema(maptype="china")
    .add("geo", [list(z) for z in zip(Faker.provinces, Faker.values())])
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(),
        title_opts=opts.TitleOpts(title="Grid-Geo-Bar"),
    )
)
# Connect above two graphs
(
    Grid(init_opts=opts.InitOpts(width='630px'))
    .add(bar, grid_opts=opts.GridOpts(pos_top="50%", pos_right="75%"))
    .add(geo, grid_opts=opts.GridOpts(pos_left="60%"))
    .render_notebook()
)


### 4)Parallel category

In [6]:
# Generte Fake values
data=[]
for i in range(14):
    data.append(Faker.values()+['Light pollution'])
(
    Parallel(init_opts=opts.InitOpts(width='630px'))
    .add_schema(
        [
            opts.ParallelAxisOpts(dim=0, name="data"),
            opts.ParallelAxisOpts(dim=1, name="AQI"),
            opts.ParallelAxisOpts(dim=2, name="PM2.5"),
            opts.ParallelAxisOpts(dim=3, name="PM10"),
            opts.ParallelAxisOpts(dim=4, name="CO"),
            opts.ParallelAxisOpts(dim=5, name="NO2"),
            opts.ParallelAxisOpts(dim=6, name="CO2"),
            opts.ParallelAxisOpts(
                dim=7,
                name="level",
                type_="category",
                data=["excellent", "good", "Light pollution", 
                      "Moderately polluted", "Heavy pollution", "Serious pollution"],
            ),
        ]
    )
    .add("parallel", data)
    .set_global_opts(title_opts=opts.TitleOpts(title="Parallel-Category"))
    .render_notebook()
)
